In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from collections import Counter
import time
import sys

In [3]:
df = pd.read_csv("../input/all_articles_stem.csv")

In [5]:
import json

hw = pd.read_json("./huawei.json", orient="index")
hw.head()

,clust,date,description,sourceName,title,url
0,2,2019-03-04,Huawei says Chinese law prohibits the governme...,wired,US Lawyers Don’t Buy Huawei’s Argument on Chin...,https://www.wired.com/story/us-lawyers-dont-bu...
1,0,2019-03-01,"Late last year, Huawei CFO Meng Wanzhou was ar...",techcrunch,Canadian government allows Huawei CFO’s US ext...,http://techcrunch.com/2019/03/01/canadian-gove...
2,0,2019-03-04,Huawei’s already proven that a ban in the worl...,techcrunch,Huawei reportedly plans to sue US government o...,http://techcrunch.com/2019/03/04/huawei-report...
3,1,2019-03-07,Huawei has decided to go on the legal offence ...,techcrunch,Huawei is suing the the US government over “un...,http://techcrunch.com/2019/03/06/huawei-sues-u...
4,0,2019-03-01,Huawei’s ongoing battle with the U.S. governme...,gizmodo,Huawei's Wild Week Ends With Canada Approving ...,https://gizmodo.com/huaweis-wild-week-ends-wit...


In [79]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import coo_matrix

# Remove Stop[words
def remove_sw(x):
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize

    example_sent = x.lower()
    
    custom_stopwords_list = [
        "new york times", #publication
        "breitbart", #publication
        "briefing",
        "evening",
        "yahoo",
        "news",
        "castanet",
        "says",
        "cnn",
        "fox",
        "bbc",
        "also",
        "said",
        "mr",
        "mrs",
        "im",
        "would",
        "could",
        "should",
        "first",
        "like",
        "dont",
        "wont",
        "get",
        "going"
    ]
    for x in custom_stopwords_list : 
        example_sent = example_sent.replace(x, "")
    
    stop_words = set(stopwords.words('english'))
    stop_words = [x.lower() for x in stop_words]
    word_tokens = word_tokenize(example_sent)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return " ".join(filtered_sentence)

def get_top_n_bigrams(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    #vec = CountVectorizer(max_df=0.85, ngram_range=(1,3)).fit(corpus)
    vectorizer = TfidfVectorizer(ngram_range=(1,2))
    vec = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names()
    sorted_items=sort_coo(vec.tocoo())
    return extract_topn_from_vector(feature_names, sorted_items)

#Function for sorting tf_idf in descending order
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=5):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
    
    return feature_vals
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
#     results= {}
#     for idx in range(len(feature_vals)):
#         results[feature_vals[idx]]=score_vals[idx]
    
#     return results

In [73]:
hw1 = hw.loc[hw['clust'] == 1]
hw1["desc2"] =  hw1["description"].apply(lambda x : remove_sw(x))
hw1["title2"] =  hw1["title"].apply(lambda x : remove_sw(x))
hw1.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,clust,date,description,sourceName,title,url,desc2,title2
3,1,2019-03-07,Huawei has decided to go on the legal offence ...,techcrunch,Huawei is suing the the US government over “un...,http://techcrunch.com/2019/03/06/huawei-sues-u...,huawei decided go legal offence united states ...,huawei suing us government “ unconstitutional ...
8,1,2019-02-19,Amid a firestorm of legal and political salvoe...,gizmodo,Huawei CEO Says U.S. Just Isn't That Important...,https://gizmodo.com/huawei-ceo-says-u-s-just-i...,amid firestorm legal political salvoes fired h...,"huawei ceo u.s. n't portant , pinky-swears spy..."
9,1,2019-02-19,The founder of Chinese electronics giant Huawe...,mashable,Huawei founder speaks amid pressure: 'The U.S....,https://mashable.com/article/huawei-founder-bb...,founder chinese electronics giant huawei n't n...,huawei founder speaks amid pressure : 'the u.s...
14,1,2019-03-01,"The Canadian government, as expected, said on ...",reuters,Canada to allow extradition case against Huawe...,https://www.reuters.com/article/us-usa-china-h...,"canadian government , expected , friday allow ...",canada allow extradition case huawei executive
18,1,2019-02-19,Ren Zhengfei tells the BBC the world needs Hua...,cnet,Huawei founder says 'there's no way the US can...,https://www.cnet.com/news/huawei-founder-says-...,ren zhengfei tells world needs huawei 's `` ad...,huawei founder 'there 's way us crush us ' - cnet


In [74]:
print(get_top_n_bigrams(list(hw1["title2"])))
# print(get_top_n_trigrams(list(hw1["title2"]))[:10])
# print(get_top_n_quadgrams(list(hw1["title2"]))[:10])

['deal', 'us', 'us huawei', 'us', 'sues canada', 'huawei exec', 'exec sues', 'exec', 'detention', 'canada detention']


In [75]:
hw2 = hw.loc[hw['clust'] == 2]
hw2["title2"] =  hw2["title"].apply(lambda x : remove_sw(x))
hw2["desc2"] =  hw2["description"].apply(lambda x : remove_sw(x))
hw2.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,clust,date,description,sourceName,title,url,title2,desc2
0,2,2019-03-04,Huawei says Chinese law prohibits the governme...,wired,US Lawyers Don’t Buy Huawei’s Argument on Chin...,https://www.wired.com/story/us-lawyers-dont-bu...,us lawyers ’ buy huawei ’ argument chinese hac...,huawei chinese law prohibits government orderi...
6,2,2019-02-27,"“Prism, prism on the wall, who’s the most trus...",gizmodo,Huawei Now Just Trolling U.S. by Dredging Up E...,https://gizmodo.com/huawei-now-just-trolling-u...,huawei trolling u.s. dredging edward snowden,"“ prism , prism wall , ’ trustworthy ? ” huawe..."
7,2,2019-03-04,Chinese telecom firm Huawei has recently come ...,gizmodo,Chinese Tech Giant Huawei Plans to Sue the US ...,https://gizmodo.com/chinese-tech-giant-huawei-...,chinese tech giant huawei plans sue us governm...,chinese telecom firm huawei recently come inte...
10,2,2019-03-04,Protesting outside the Supreme Court of Britis...,nytimes,"Meng Wanzhou, Seen as ‘Poor Little Rich Girl,’...",https://www.nytimes.com/2019/03/04/world/canad...,"meng wanzhou , seen ‘ poor little rich girl , ...",protesting outside supreme court british colum...
11,2,2019-03-07,The Chinese firm is facing international scrut...,bbc,Huawei sues US government over product ban,https://www.bbc.co.uk/news/business-47478587,huawei sues us government product ban,chinese firm facing international scrutiny tie...


In [76]:
print(get_top_n_bigrams(list(hw2["title2"])))

['world controversial', 'world', 'controversial company', 'controversial', 'state', 'company', 'us', 'top asian', 'gmt', 'asian 01']


In [85]:
import json
def getTags(path_to_json, name):

    hw = pd.read_json(path_to_json, orient="index")
    clustTags = {}
    for clust in hw['clust'].unique():
        hwClust = hw.loc[hw['clust'] == clust]
        hwClust["title2"] =  hwClust["title"].apply(lambda x : remove_sw(x))
        clustTags[str(clust)] = get_top_n_bigrams(list(hwClust["title2"]))
    
    jsonName = "./" + str(name) +'-tags.json'
    with open(jsonName, 'w') as fp:
        json.dump(clustTags, fp)

getTags("./huawei.json", "huawei")
getTags("./shutdown.json", "shutdown")
getTags("./venezuela.json", "venezuela")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind